In [1]:
import os
import argparse
import json
from dataclasses import dataclass, field
from pathlib import Path
from typing import Generator
import re
import toml

os.environ["PATH"] += ':/home/ex-anastasia/Premise-Retrieval/.venv/bin'
os.environ["PATH"] += ':/home/ex-anastasia/.elan/bin'

Here, we test out various ways of evaluating the model, including the classic evaluation on the lean-dojo benchmark (for f2f it should be the same?) & the evals used in Premise-Retrieval. 

In [2]:
# load the lean-dojo dataset from mathlib_handler_benchmark

DST_DIR = Path("/home/ex-anastasia/Premise-Retrieval/mathlib_handler_benchmark_410/")

test_path = DST_DIR / "random/test.jsonl"
lines_test = list(test_path.open())
one_line_test = json.loads(lines_test[0])

# original state + goal 
print(f"Original state and goal: {one_line_test['tactics'][0]['state_before']}")
proof_steps = [el['tactic'] for el in one_line_test['tactics']]
print(f"Proof: {proof_steps}")
print(f"Premises: {[el['premises'] for el in one_line_test['tactics']]}")

url = one_line_test['url']
full_name = one_line_test['full_name']
short_name = full_name.split(".")[-1]
print(f"URL: {url}")
print(f"Full name: {full_name}")
print(f"Short name: {short_name}")

Original state and goal: R : Type u_1
α : Type u_2
β : Type u_3
δ : Type u_4
γ : Type u_5
ι : Type u_6
m0 : MeasurableSpace α
inst✝¹ : MeasurableSpace β
inst✝ : MeasurableSpace γ
μ μ₁ μ₂ μ₃ ν ν' ν₁ ν₂ : Measure α
s s' t : Set α
h : s ⊆ t
⊢ (μ.restrict t).restrict s = μ.restrict s
Proof: ['ext1 u hu', 'rw [restrict_apply hu, restrict_apply hu, restrict_eq_self]', 'exact inter_subset_right.trans h']
Premises: [[], [{'full_name': 'MeasureTheory.Measure.restrict_apply', 'def_path': 'Mathlib/MeasureTheory/Measure/Restrict.lean', 'def_pos': [66, 8], 'def_end_pos': [66, 22]}, {'full_name': 'MeasureTheory.Measure.restrict_eq_self', 'def_path': 'Mathlib/MeasureTheory/Measure/Restrict.lean', 'def_pos': [111, 8], 'def_end_pos': [111, 24]}], [{'full_name': 'Set.inter_subset_right', 'def_path': 'Mathlib/Data/Set/Basic.lean', 'def_pos': [764, 8], 'def_end_pos': [764, 26]}]]
URL: Mathlib/MeasureTheory/Measure/Restrict.lean
Full name: MeasureTheory.Measure.restrict_restrict_of_subset
Short name: restr

Running lean-dojo on the dataset

In [3]:
from lean_dojo import LeanGitRepo, trace, Theorem, Dojo, TacticState

In [4]:
repo_url = 'https://github.com/leanprover-community/mathlib'
repo = LeanGitRepo(
    "https://github.com/leanprover-community/mathlib4",
    "29dcec074de168ac2bf835a77ef68bbe069194c5",
)
repo.get_config("lean-toolchain")

{'content': 'leanprover/lean4:v4.10.0-rc1\n'}

In [5]:
traced_repo = trace(repo)

2025-03-19 18:32:37.200 | INFO     | lean_dojo.data_extraction.trace:trace:248 - Loading the traced repo from /home/ex-anastasia/.cache/lean_dojo/leanprover-community-mathlib4-29dcec074de168ac2bf835a77ef68bbe069194c5/mathlib4
2025-03-19 18:32:38,956	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
100%|██████████| 5674/5674 [10:52<00:00,  8.70it/s]  


In [6]:
print(len(traced_repo.traced_files))

5674


In [7]:
traced_file = traced_repo.get_traced_file("Mathlib/Algebra/BigOperators/Pi.lean")
print(traced_file)
traced_file.get_premise_definitions()

theorem = Theorem(repo, "Mathlib/Algebra/BigOperators/Pi.lean", "pi_eq_sum_univ")
dojo, new_state = Dojo(theorem).__enter__()

TracedFile(root_dir=PosixPath('/home/ex-anastasia/.cache/lean_dojo/leanprover-community-mathlib4-29dcec074de168ac2bf835a77ef68bbe069194c5/mathlib4'), repo=LeanGitRepo(url='/home/ex-anastasia/.cache/lean_dojo/leanprover-community-mathlib4-29dcec074de168ac2bf835a77ef68bbe069194c5/mathlib4', commit='29dcec074de168ac2bf835a77ef68bbe069194c5'), lean_file=LeanFile(path=PosixPath('Mathlib/Algebra/BigOperators/Pi.lean')))


In [10]:
for it in range(10):
    one_line_test = json.loads(lines_test[it])

    url = one_line_test['url']
    full_name = one_line_test['full_name']
    short_name = full_name.split(".")[-1]
    try: 
        theorem = Theorem(repo, url, full_name)

        dojo, new_state = Dojo(theorem).__enter__()

        proof_steps = [el['tactic'] for el in one_line_test['tactics']]
        for proof_step in proof_steps: 
            new_state = dojo.run_tac(new_state, proof_step)
            print(new_state)
    except: 
        print('AST file not found for theorem: ' + short_name)

TacticState(pp="case h\nR : Type u_1\nα : Type u_2\nβ : Type u_3\nδ : Type u_4\nγ : Type u_5\nι : Type u_6\nm0 : MeasurableSpace α\ninst✝¹ : MeasurableSpace β\ninst✝ : MeasurableSpace γ\nμ μ₁ μ₂ μ₃ ν ν' ν₁ ν₂ : Measure α\ns s' t : Set α\nh : s ⊆ t\nu : Set α\nhu : MeasurableSet u\n⊢ ((μ.restrict t).restrict s) u = (μ.restrict s) u", id=1, message='')
TacticState(pp="case h.h\nR : Type u_1\nα : Type u_2\nβ : Type u_3\nδ : Type u_4\nγ : Type u_5\nι : Type u_6\nm0 : MeasurableSpace α\ninst✝¹ : MeasurableSpace β\ninst✝ : MeasurableSpace γ\nμ μ₁ μ₂ μ₃ ν ν' ν₁ ν₂ : Measure α\ns s' t : Set α\nh : s ⊆ t\nu : Set α\nhu : MeasurableSet u\n⊢ u ∩ s ⊆ t", id=2, message='')
ProofFinished(tactic_state_id=3, message='')
LeanError(error="tactic 'introN' failed, insufficient number of binders\ncase nil\nα : Type u\nβ : Type v\nl l₁ l₂ : List α\nr : α → α → Prop\na b : α\n⊢ [].Nodup ↔ ∀ (a : α), ¬[a, a] <+ []\ntactic 'introN' failed, insufficient number of binders\ncase cons\nα : Type u\nβ : Type v\nl✝ l

In [ ]:
# Test with an LLM model

import torch
import transformers
